In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score

In [2]:
labels = pd.read_csv("../../data/scene_labels.csv")

In [3]:
with open('../../audio_features/feat_dict_librosa_lld.pickle', 'rb') as f:
    librosa_audio_features = pickle.load(f, encoding='latin1')

In [4]:
labels.head(5)

,SCENE,KEY,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,BBT,0.0,NONE
1,1_10009,1_10009_u,BBT,0.0,NONE
2,1_1001,1_1001_u,BBT,0.0,NONE
3,1_1003,1_1003_u,BBT,1.0,PRO
4,1_10190,1_10190_u,BBT,0.0,NONE


#### Perform mean, median, max, min and sum pooling on audio feature data

In [5]:
def get_audio_mean_pool(audio) -> np.ndarray:
        return np.array([np.mean(feature_vector, axis=1) for feature_vector in audio])
    
def get_audio_median_pool(audio) -> np.ndarray:
        return np.array([np.median(feature_vector, axis=1) for feature_vector in audio])
    
def get_audio_max_pool(audio) -> np.ndarray:
        return np.array([np.max(feature_vector, axis=1) for feature_vector in audio])

def get_audio_min_pool(audio) -> np.ndarray:
        return np.array([np.min(feature_vector, axis=1) for feature_vector in audio])

def get_audio_sum_pool(audio) -> np.ndarray:
        return np.array([np.sum(feature_vector, axis=1) for feature_vector in audio])

In [6]:
model_data = pd.DataFrame(columns=['audio_feature','sarcasm','sarcasm_type'])

for index, row in labels.iterrows():
    audio_key = row["SCENE"] + "_u.wav"
    model_data = model_data.append({'audio_feature': librosa_audio_features[audio_key],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"]},
                                  ignore_index=True)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(
    model_data['audio_feature'],
    model_data['sarcasm_type'],
    train_size=0.8, 
    test_size=0.2, 
    random_state=0, 
    shuffle=True,
    stratify=model_data['sarcasm_type'])

X_train_mean = get_audio_mean_pool(X_train)
X_test_mean = get_audio_mean_pool(X_test)

X_train_median = get_audio_median_pool(X_train)
X_test_median = get_audio_median_pool(X_test)

X_train_max = get_audio_max_pool(X_train)
X_test_max = get_audio_max_pool(X_test)

X_train_min = get_audio_min_pool(X_train)
X_test_min = get_audio_min_pool(X_test)

X_train_sum = get_audio_sum_pool(X_train)
X_test_sum = get_audio_sum_pool(X_test)

print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))

print("Train mean: ",X_train_mean.shape, Y_train.shape,
      "Test mean: ",(X_test_mean.shape, Y_test.shape))

Train:  (961,) (961,) Test:  ((241,), (241,))
Train mean:  (961, 690) (961,) Test mean:  ((241, 690), (241,))


In [8]:
svm_clf = svm.SVC(random_state=0, kernel = "rbf", gamma = "scale", class_weight = "balanced")
C = [0.001, 0.01, 0.1, 1, 10]
gscv_clf = GridSearchCV(
    estimator=svm_clf, 
    param_grid=dict(C=C),
    n_jobs=-1, 
    cv = 10, 
    scoring = 'f1_macro', 
    refit = True)

gscv_clf.fit(X_train_mean, Y_train)
Y_test_pred = gscv_clf.predict(X_test_mean)
report = classification_report(Y_test, Y_test_pred)
print(report)

              precision    recall  f1-score   support

         EMB       0.05      0.12      0.07        17
         ILL       0.20      0.11      0.14        36
         LIK       0.00      0.00      0.00         1
        NONE       0.69      0.53      0.60       120
         PRO       0.34      0.16      0.22        67

    accuracy                           0.34       241
   macro avg       0.26      0.19      0.21       241
weighted avg       0.47      0.34      0.39       241



In [9]:
gscv_clf.best_estimator_

SVC(C=10, class_weight='balanced', random_state=0)

In [10]:
svm_clf = svm.SVC(random_state=0, kernel = "rbf", gamma = "scale", class_weight = "balanced")
C = [0.001, 0.01, 0.1, 1, 10]
gscv_clf = GridSearchCV(
    estimator=svm_clf, 
    param_grid=dict(C=C),
    n_jobs=-1, 
    cv = 10, 
    scoring = 'f1_macro', 
    refit = True)

gscv_clf.fit(X_train_median, Y_train)
Y_test_pred = gscv_clf.predict(X_test_median)
report = classification_report(Y_test, Y_test_pred)
print(report)

              precision    recall  f1-score   support

         EMB       0.05      0.12      0.07        17
         ILL       0.11      0.14      0.12        36
         LIK       0.00      0.00      0.00         1
        NONE       0.69      0.39      0.50       120
         PRO       0.41      0.18      0.25        67

    accuracy                           0.27       241
   macro avg       0.25      0.17      0.19       241
weighted avg       0.48      0.27      0.34       241



In [11]:
gscv_clf.best_estimator_

SVC(C=10, class_weight='balanced', random_state=0)

In [12]:
svm_clf = svm.SVC(random_state=0, kernel = "rbf", gamma = "scale", class_weight = "balanced")
C = [0.001, 0.01, 0.1, 1, 10]
gscv_clf = GridSearchCV(
    estimator=svm_clf, 
    param_grid=dict(C=C),
    n_jobs=-1, 
    cv = 10, 
    scoring = 'f1_macro', 
    refit = True)

gscv_clf.fit(X_train_max, Y_train)
Y_test_pred = gscv_clf.predict(X_test_max)
report = classification_report(Y_test, Y_test_pred)
print(report)

              precision    recall  f1-score   support

         EMB       0.05      0.29      0.09        17
         ILL       0.11      0.03      0.04        36
         LIK       0.00      0.00      0.00         1
        NONE       0.62      0.45      0.52       120
         PRO       0.29      0.15      0.20        67

    accuracy                           0.29       241
   macro avg       0.22      0.18      0.17       241
weighted avg       0.41      0.29      0.33       241



In [13]:
gscv_clf.best_estimator_

SVC(C=10, class_weight='balanced', random_state=0)

In [14]:
svm_clf = svm.SVC(random_state=0, kernel = "rbf", gamma = "scale", class_weight = "balanced")
C = [0.001, 0.01, 0.1, 1, 10]
gscv_clf = GridSearchCV(
    estimator=svm_clf, 
    param_grid=dict(C=C),
    n_jobs=-1, 
    cv = 10, 
    scoring = 'f1_macro', 
    refit = True)

gscv_clf.fit(X_train_min, Y_train)
Y_test_pred = gscv_clf.predict(X_test_min)
report = classification_report(Y_test, Y_test_pred)
print(report)

              precision    recall  f1-score   support

         EMB       0.00      0.00      0.00        17
         ILL       0.16      0.19      0.17        36
         LIK       0.00      0.00      0.00         1
        NONE       0.60      0.48      0.54       120
         PRO       0.41      0.27      0.32        67

    accuracy                           0.34       241
   macro avg       0.23      0.19      0.21       241
weighted avg       0.44      0.34      0.38       241



In [15]:
gscv_clf.best_estimator_

SVC(C=10, class_weight='balanced', random_state=0)

In [16]:
svm_clf = svm.SVC(random_state=0, kernel = "rbf", gamma = "scale", class_weight = "balanced")
C = [0.001, 0.01, 0.1, 1, 10]
gscv_clf = GridSearchCV(
    estimator=svm_clf, 
    param_grid=dict(C=C),
    n_jobs=-1, 
    cv = 10, 
    scoring = 'f1_macro', 
    refit = True)

gscv_clf.fit(X_train_sum, Y_train)
Y_test_pred = gscv_clf.predict(X_test_sum)
report = classification_report(Y_test, Y_test_pred)
print(report)

              precision    recall  f1-score   support

         EMB       0.05      0.12      0.07        17
         ILL       0.17      0.17      0.17        36
         LIK       0.02      1.00      0.04         1
        NONE       0.67      0.52      0.58       120
         PRO       0.29      0.10      0.15        67

    accuracy                           0.32       241
   macro avg       0.24      0.38      0.20       241
weighted avg       0.44      0.32      0.36       241



In [17]:
gscv_clf.best_estimator_

SVC(C=10, class_weight='balanced', random_state=0)